In [1]:
import os


In [2]:
%pwd

'e:\\ML Folder\\endtoendsalaryprediction_MLOps\\research'

In [3]:
os.chdir("E:\\ML Folder\\endtoendsalaryprediction_MLOps\\research")

In [4]:
os.chdir("../")

In [5]:
%pwd

'e:\\ML Folder\\endtoendsalaryprediction_MLOps'

In [6]:
from src.salaryprediction_mlops.constants import *
from src.salaryprediction_mlops.utils.common import read_yaml,create_directories

In [7]:
from dataclasses import dataclass
from pathlib import Path

In [8]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url:str
    local_data_file:Path
    unzip_dir:Path


In [9]:
class ConfigurationManager:
      def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
            self.config = read_yaml(config_filepath)
            self.param = read_yaml(param_filepath)
            self.schema = read_yaml(schema_filepath)

            create_directories([self.config.artifacts_root],verbose=True)

      def get_data_ingestion_config(self)->DataIngestionConfig:
            config = self.config.data_ingestion

            create_directories([config.root_dir])

            data_ingestion_config = DataIngestionConfig(
                  root_dir = config.root_dir,
                  source_url = config.source_url,
                  local_data_files = config.local_data_files,
                  unzip_dir = config.unzip_dir
            )

            return data_ingestion_config

    

        

    

In [10]:
import os
import urllib.request as request
import zipfile
from src.salaryprediction_mlops.logging import logger
from src.salaryprediction_mlops.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config=config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,header=request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: \n {headers}")
        else:
            logger.info(f"file already exists of size : {get_size(Path(self.config.local_data_files))}")

    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-21 11:16:21,770:INFO:common:reading the yaml file from config\config.yaml]
[2025-01-21 11:16:21,774:INFO:common:reading the yaml file from params.yaml]


EnsureError: Return value of <function read_yaml at 0x000001BCE3A165F0> of type <class 'type'> does not match annotation type <class 'box.config_box.ConfigBox'>

In [19]:
from ensure import ensure_annotations
from box import ConfigBox
from box.exceptions import BoxValueError
import yaml
@ensure_annotations
def read_yaml(path_to_yaml:Path)->ConfigBox:
    try:
        with open(path_to_yaml) as yaml_file:
            content=yaml.safe_load(yaml_file)
            logger.info(f"reading the yaml file from {path_to_yaml}")
            return ConfigBox(content)
    except BoxValueError:
        return ValueError
    except Exception as e:
        raise e

In [20]:
read_yaml(CONFIG_FILE_PATH)

[2025-01-21 11:22:17,759:INFO:2929098797:reading the yaml file from config\config.yaml]


ConfigBox({'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/lavaniarajeev001/datasets/blob/main/salary_prediction.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}})